In [5]:
import tensorflow as tf
tf.enable_eager_execution()

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import functools

In [78]:
obs = tf.random_normal([2, 10])
r = tf.random_normal([2, 1])

@threadpool
def choose_action(obs, r, fn):
    time.sleep(0.1)
    return fn(tf.concat([obs, r], axis=1))

a = choose_action(obs, r, tf.keras.layers.Dense(12))
a.result()

<tf.Tensor: id=979, shape=(2, 12), dtype=float32, numpy=
array([[ 0.2686407 ,  0.27760425, -0.15348823,  0.4542377 , -0.03909007,
        -0.09310319, -1.4964381 ,  0.4523476 , -0.8409841 , -1.6969504 ,
        -0.49972126, -0.04047663],
       [-0.03217527, -0.18008229,  0.23739257, -0.20665997,  1.7399492 ,
        -0.40579495, -0.14870209,  0.9199598 , -0.37035343,  0.29321948,
        -1.3538345 , -1.3721434 ]], dtype=float32)>

In [79]:
_DEFAULT_POOL = ThreadPoolExecutor()

def threadpool(f, executor=None):
    # from bj0 on SE 
    # https://stackoverflow.com/questions/6893968/how-to-get-the-return-value-from-a-thread-in-python
    @functools.wraps(f)
    def wrap(*args, **kwargs):
        return (executor or _DEFAULT_POOL).submit(f, *args, **kwargs)

    return wrap

In [80]:
class Net():
    def __init__(self):
        self.nn = tf.keras.layers.Dense(12)
    
        self.old_obs = obs
        self.old_r = r
        self.action = choose_action(obs, r, self.nn)
        
    def __call__(self, obs, r):
        # NOTE because we dont use this it doesnt block
        # and has time to be calculated
        self.old_action = self.action.result()  
        
        self.action = choose_action(obs, r, self.nn)
        
        self.old_obs = obs
        self.old_r = r
                
        return self.old_action

In [85]:
net = Net()
for _ in range(10):
    obs = tf.random_normal([2, 10])
    r = tf.random_normal([2, 1])
    
    start = time.time()
    
    a = net(obs, r)
#     print(time.time() - start)
    time.sleep(0.1)  # time for the env to do something
    
    print(time.time() - start)

0.20319342613220215
0.10006284713745117
0.10015726089477539
0.10016489028930664
0.10016107559204102
0.1001741886138916
0.10023379325866699
0.10009264945983887
0.10020565986633301
0.10016345977783203
